<a href="https://colab.research.google.com/github/liuxx479/NGphotoz/blob/master/prep_mass_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pylab import *
from scipy.stats import norm
from scipy.stats import uniform
from astropy.io import fits
from astropy import units as u
import os
sys.modules["mpi4py"] = None
from lenstools import ConvergenceMap
from IPython.display import Image
%matplotlib inline

### goals of this notebook
##### 1) compute the nubmer density per z bin, using the fiducial photo-z (sigma_z=0, or step function was used)
##### 2) add noise as GRF to each redshift
##### 3) smooth the maps
##### 4) define the bin edges
##### 5) compute peaks, minima, PDF, MFs, moments
##### 6) turn this into a python code

In [ ]:
# !git add prep_map_stats.ipynb; git commit -m 'verified sigma_kappa = sqrt(2) sigma_e '; git push
!git add map_stats.py; git commit -m 'map_stats.py test pass; ready for mass production'; git push

In [ ]:
##### number density per redshift bins
def Pz_norm(zp_support, pz):
  return pz/np.trapz(pz, zp_support)

def Pz_norm_tomo(zp_support, pz):
  return array([Pz_norm(zp_support, ipz) for ipz in pz])

zarr=np.linspace(0,4, 501)
delta_z=0.5

tomo_centers=np.arange(0.5, 3, delta_z)
tomo_widths=np.ones(5)*delta_z
tomo_edges = np.array([tomo_centers-delta_z/2, tomo_centers+delta_z/2]).T

z0_SRD, alpha_SRD = 0.11, 0.68
Pz_SRD_unnorm = lambda z: z**2 * exp(-(z/z0_SRD)**alpha_SRD)
Pz_SRD_norm = lambda z: Pz_norm(z, Pz_SRD_unnorm(z))

plot(zarr, Pz_SRD_norm(zarr), lw=2, label='LSST gold (SRD2018 Y10 p53)')
xlabel('z')
ylabel('Pz')
legend()
show()

In [ ]:
############ fiducial model: pz_true.txt 
### zbias=0, sigma_z=0, outlier=0; it should have sharp cut off at the bin edges ([0.25, 0.75], [0.75, 1.25]...)
####### LSST SRD Y10 numbers page 53
######## SRD: https://arxiv.org/abs/1809.01669
## neff: (z0, α) = (0.13, 0.78) for Y1 and (0.11, 0.68) for Y10
## neff = 10 and 27 arcmin−2 as the lensing neff in Y1 and Y10
## however, according to the original Chang 2013 paper, unmasked neff=37
sigma_e = 0.26 ## per component, do I need a sqrt(2) to get both components? (I didn't do this in 2014..)
neff = 37 ## chang+2013 n=46, for fiducial case
#ngal = 46
ngal = neff

### 1) compute the nubmer density per z bin

In [ ]:
ngal_tomo = np.zeros(5)
Pz_norm_factor = np.trapz(Pz_SRD_unnorm(zarr),zarr)
print (Pz_norm_factor)

for i in range(5):
    iz0,iz1=tomo_edges[i]
    izarr=zarr[where((zarr>iz0)&(zarr<iz1))]
    ngal_tomo[i]= ngal * np.trapz(Pz_SRD_unnorm(izarr), izarr) / Pz_norm_factor

In [ ]:
print ('number of galaxies per redshift bin \n(ngal=%i, %s galaxies in 5 bins):'%(ngal, sum(ngal_tomo)))
for i in range(5):
    print ('z=%s, ngal=%.2f /arcmin^2'%(tomo_edges[i], ngal_tomo[i]))

### 2) add noise as GRF to each redshift

In [ ]:
###### get a set of 5 test maps
#test_dir = '/global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/06_f/'
# std: [0.007314169, 0.019146424, 0.031151935, 0.041499652, 0.050280314]

##### change to fiducial model, so I can get the std for the maps
test_dir = '/global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/fid_f/'
os.listdir(test_dir)[:10]
fnames = [test_dir+'kappa_LSST-SRD_tomo%i_cone1.fits'%(i) for i in range(1,6)]
test_maps = array([fits.open(ifn)[0].data for ifn in fnames])
istds = [std(it) for it in test_maps]
print (istds)

In [ ]:
#### plot original kappa maps
figure(figsize=(16,2))
for i in range(5):
    subplot(1,5,i+1)
    istd=istds[i]
    imshow(test_maps[i], vmin=-3*istd, vmax=3*istd, origin='lower', extent=[0,10,0,10])
    title('noiseless z=%.1f'%(tomo_centers[i]))
    colorbar()

In [ ]:
### white noise maps for shape noise
map_side_deg = 10*u.degree
pixel_angular_side = map_side_deg / test_maps[0].shape[0]

sigma_pix_arr =[ (sigma_e / (pixel_angular_side * sqrt(ingal / u.arcmin**2))).decompose().value
                for ingal in ngal_tomo]

print (map_side_deg, test_maps[0].shape[0])
print (pixel_angular_side.to (u.arcmin))
print (sigma_pix_arr)

np.random.seed(2)
noise_maps = [np.random.normal(loc=0.0, scale=sigma_pix_arr[i], size=test_maps[0].shape) 
              for i in range(5)]

# #Generate shape noise
#sigma = ((0.15 + 0.035*z) / (pixel_angular_side * np.sqrt(ngal))).decompose().value
# np.random.seed(seed)
# noise_map = np.random.normal(loc=0.0,scale=sigma,size=self.shape) 

In [ ]:
## turn it into convergencemap instance
conv_noiseless_maps = [ConvergenceMap(data=test_maps[i], angle=map_side_deg) for i in range(5)]
conv_maps = [ConvergenceMap(data=test_maps[i]+noise_maps[i], angle=map_side_deg) for i in range(5)]

In [ ]:
istds_noisy = [conv_maps[i].std() for i in range(5)]
print (istds_noisy)

In [ ]:
figure(figsize=(16,2))
for i in range(5):
    subplot(1,5,i+1)
    istd=istds_noisy[i]
    imshow(conv_maps[i].data, vmin=-3*istd, vmax=3*istd, origin='lower', extent=[0,10,0,10])
    title('z=%.1f \n ngal=%.2f'%(tomo_centers[i], ngal_tomo[i]))
    colorbar()

### 3) smooth the maps

In [ ]:
theta_g = 5 #arcmin
smoothed_conv_maps= [conv_maps[i].smooth(theta_g*u.arcmin) for i in range(5)]

In [ ]:
istds_smoothed = [smoothed_conv_maps[i].std() for i in range(5)]
print (istds_smoothed)

In [ ]:
figure(figsize=(16,2))
for i in range(5):
    subplot(1,5,i+1)
    istd=istds_smoothed[i]
    imshow(smoothed_conv_maps[i].data, vmin=-3*istd, vmax=3*istd, origin='lower', extent=[0,10,0,10])
    title('z=%.1f  ngal=%.2f \n smooth=%s arcmin'%(tomo_centers[i], ngal_tomo[i], theta_g))
    colorbar()

### 4) define the bin edges

<img src="theory_noise_smoothed.jpg" width="400">

In [ ]:
######### the std of noise maps
istds_noise_smoothed = array([ConvergenceMap(data=noise_maps[i], 
                                             angle=map_side_deg).smooth(theta_g*u.arcmin,mode='wrap').std() 
                            for i in range(5)])

In [ ]:
# this is from Coulton2019, not sure where is the log(2) factor from
# const_n = 1/sqrt(4*log(2)*pi)#0.3388303758015525 

# this matches yang2011, with their 1 arcmin = my/usual 1/sqrt(2) arcmin, and seem correct
const_n = 1/sqrt(4*pi)

## theta_g in unit of 1 arcmin, ngal in unit of 1/armin^2
sigma_smooth = lambda ngal, theta_g: const_n* sigma_e/(theta_g * sqrt(ngal))
sigma_n_theory = sigma_smooth (array(ngal_tomo), theta_g)

print ('std for smoothed maps:')
print (array(istds_smoothed), 'sim (white noise+kappa)')
print (istds_noise_smoothed, 'sim (white noise)')
print (sigma_n_theory, 'theory (white noise)')
print (sigma_n_theory/istds_noise_smoothed, 'theory/sim')

In [ ]:
### make sure smoothing is not done in place - pass
test_map = ConvergenceMap(data=rand(4,4), angle=4*u.arcmin)
print (test_map.data)
smoothed_test_maps = test_map.smooth(1*u.arcmin)
print (smoothed_test_maps.data)
print (test_map.data)

In [ ]:
sigma_bin_edges = linspace(-5,5,21)
print ('bin edges (sigma):\n',sigma_bin_edges)

kappa_bin_edges = kron(istds_smoothed,sigma_bin_edges).reshape(5,-1)
#print ('bin edges (kappa):\n', kappa_bin_edges)

### 5) compute peaks, minima, PDF, MFs, moments

In [ ]:
l_edges = logspace(log10(40),log10(4000),21)
ps_test = [conv_maps[i].powerSpectrum(l_edges) for i in range(5)]

In [ ]:
for i in range(5):
    loglog(ps_test[i][0], ps_test[i][1], label='z=%s'%(tomo_centers[i]))
xlabel('ell')
ylabel('Clkk')
legend()

In [ ]:
peaks_test = [smoothed_conv_maps[i].peakCount(kappa_bin_edges[i]) for i in range(5)]

In [ ]:
pdf_test = [smoothed_conv_maps[i].pdf(kappa_bin_edges[i]) for i in range(5)]

In [ ]:
minima_test = [ConvergenceMap(data=-smoothed_conv_maps[i].data, 
                              angle=map_side_deg).peakCount(kappa_bin_edges[i]) for i in range(5)]

In [ ]:
mf_test = [smoothed_conv_maps[i].minkowskiFunctionals(kappa_bin_edges[i]) for i in range(5)]

In [ ]:
#moments_test = [smoothed_conv_maps[i].moments(kappa_bin_edges[i]) for i in range(5)]

In [ ]:
f,axes=subplots(2,3,figsize=(14,6))
for i in range(5):
    axes[0][0].plot(peaks_test[i][0], peaks_test[i][1], label='z=%s'%(tomo_centers[i]))
    axes[0][1].plot(minima_test[i][0], minima_test[i][1][::-1])## note need to flip the axis
    axes[0][2].plot(pdf_test[i][0], pdf_test[i][1])
    axes[1][0].plot(mf_test[i][0],mf_test[i][1])
    axes[1][1].plot(mf_test[i][0],mf_test[i][2])
    axes[1][2].plot(mf_test[i][0],mf_test[i][3])
axes[0][0].legend()
axes[1][0].set_xlabel('kappa')
axes[1][1].set_xlabel('kappa')
axes[1][2].set_xlabel('kappa')
axes[0][0].set_ylabel('N_peaks')
axes[0][1].set_ylabel('N_minima')
axes[0][2].set_ylabel('PDF')
axes[1][0].set_ylabel('V0')
axes[1][1].set_ylabel('V1')
axes[1][2].set_ylabel('V2')
plt.tight_layout()

### 6) turn this into a python code

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})

In [ ]:
!pwd

In [ ]:
## get the std for 5 z, theta_g = 1, 5, 10
Nbin=20
theta_g_arr = [1,5,10]
###### this cell takes long time to compute
smoothed_conv_maps= [[conv_maps[i].smooth(theta_g*u.arcmin) for i in range(5)]
                     for theta_g in theta_g_arr]
sigma_kappa_arr = [[imap.std() for imap in maps] for maps in smoothed_conv_maps]
savetxt('sigma_kappa.txt', sigma_kappa_arr, header='#rows=3 smoothing scales (1,5,10), cols=5 tomo bins')
!git add sigma_kappa.txt; git commit -m 'add sigma_kappa.txt'; git push

sigma_kappa_arr = genfromtxt('sigma_kappa.txt')

print (array(sigma_kappa_arr).shape)
print ('sigma_kappa for 3 smoothing, 5 tomo bins:')

for isigma in sigma_kappa_arr:
    print (isigma)

In [ ]:
savetxt('ngal_tomo.txt', ngal_tomo, header='# ngal for 5 tomographic bins')
ngal_tomo = loadtxt('ngal_tomo.txt')
print (ngal_tomo)

In [ ]:
## galaxy noise, assuming z0_SRD, alpha_SRD = 0.11, 0.68, ngal_total=46
# ngal_tomo = array([16.0651584 , 12.82188281,  6.9319586 ,  3.5123717 ,  1.66494164])

## first set bin edges
sigma_bin_edges = linspace(-5,5,Nbin+1)

kappa_bin_edges = kron(sigma_kappa_arr,sigma_bin_edges).reshape(3,5,-1)
print (kappa_bin_edges.shape)
#print ('(kappa) bin edges for 3 smoothing, 5 tomo bins:\n', kappa_bin_edges)

In [ ]:
# test_dir=/global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/
!ls /global/cscratch1/sd/jialiu/desc-sprint-raytracing/
!ls /global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/
!ls /global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/00_a/*tomo1*fits

In [ ]:
cosmos = [ '%02d_%s'%(i, j) for i in range(25) for j in ['a','f']]
cosmos += ['fid_a', 'fid_f']
cosmo_dir = '/global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/'
cosmo_fn_gen = lambda cosmo, tomo, cone: cosmo_dir+cosmo+'/kappa_LSST-SRD_tomo%i_cone%i.fits'%(tomo, cone)

### tomo runs from 1-5, cone run from 1-5, cosmo run from 0-24, 25 is the fiducial model

In [ ]:
####### test file name geneartor: pass
# cosmo_fn_gen(cosmos[0], 2, 3)
# ! ls /global/cscratch1/sd/jialiu/desc-sprint-raytracing/Cosmo_maps/00_a/kappa_LSST-SRD_tomo2_cone3.fits

In [ ]:
l_edges = logspace(log10(40),log10(4000),Nbin+1)

sigma_e=0.26
map_side_deg = 10*u.degree
map_pix = 7745
theta_g_arr = [1,5,10]

pixel_angular_side = map_side_deg / map_pix

sigma_pix_arr =[ (sigma_e / (pixel_angular_side * sqrt(ingal / u.arcmin**2))).decompose().value
                for ingal in ngal_tomo]

print (sigma_pix_arr)

## kappa_bin_edges.shape == (3,5,21)
def map_stats (cosmo, tomo, cone):
    '''for fits file fn, generate ps, peaks, minima, pdf, MFs
    fn: input file name, including full path
    tomo=1, 2,..5: int, for tomographic bins
    cone=1, 2,..5: int, for light cones'''
    fn = cosmo_fn_gen(cosmo, tomo, cone)
    imap = fits.open(fn)[0].data ## open the file
    ### add noise
    ### generate random see, such that it is the same for all cosmology
    ### but different for tomo and cone
    if cosmo[-1]=='a':
        iseed=int(cone*100+tomo)
    else: ##'f' starts with a different seed from the a cosmology
        iseed=int(1000+cone*100+tomo)
    seed(iseed)
    noise_map = np.random.normal(loc=0.0, scale=sigma_pix_arr[tomo-1], size=(map_pix, map_pix))
    kappa_map = ConvergenceMap(data=imap+noise_map, angle=map_side_deg)
    
    ### smooth the map
    smoothed_kappa_maps= [kappa_map.smooth(theta_g*u.arcmin) for theta_g in theta_g_arr]
    
    out=zeros(shape=(3, 11, Nbin)) 
    # 3 smoothing, 9 cols: ell, ps, kappa, peak, minima, pdf, v0, v1, v2
    ### compute stats
    ps_noiseless=ConvergenceMap(data=imap, angle=map_side_deg).powerSpectrum(l_edges)
    ps_unsmoothed=kappa_map.powerSpectrum(l_edges) ## power spectrum should be computed on unsmoothed maps
    
    s=0 ## smoothing scale counter
    for imap in smoothed_kappa_maps:
        kappa_bins = kappa_bin_edges[s][tomo-1] 
        ps=imap.powerSpectrum(l_edges)
        peak=imap.peakCount(kappa_bins)
        minima = ConvergenceMap(data=-imap.data, angle=map_side_deg).peakCount(kappa_bins)
        pdf=imap.pdf(kappa_bins)
        mfs=imap.minkowskiFunctionals(kappa_bins)
        out[s,0] = ps[0]
        out[s,1] = ps_noiseless[1]
        out[s,2] = ps_unsmoothed[1]
        out[s,3] = ps[1]
        out[s,4] = peak[0]
        out[s,5] = peak[1]
        out[s,6] = minima[1][::-1]
        out[s,7] = pdf[1]
        out[s,8] = mfs[1]
        out[s,9] = mfs[2]
        out[s,10] = mfs[3]
        s+=1
    return out

In [ ]:
out=map_stats(cosmos[-1],4,5)

In [ ]:
## A) 124 maps for the covariance matrix out of the 800+ I plan to do, 
## B) 10 maps per cosmology out of potentially 50, and 
## C) one map per n(z) case.

In [ ]:
f,axes=subplots(3,3,figsize=(14,9))
for s in range(3):
    axes[0][0].loglog(out[s][0],out[s][1])
    axes[0][1].loglog(out[s][0],out[s][2])
    axes[0][2].loglog(out[s][0],out[s][3],label='%i arcmin'%(theta_g_arr[s]))
    axes[1][0].plot(out[s][4],out[s][5])
    axes[1][1].plot(out[s][4],out[s][6])
    axes[1][2].plot(out[s][4],out[s][7])
    axes[2][0].plot(out[s][4],out[s][8])
    axes[2][1].plot(out[s][4],out[s][9])
    axes[2][2].plot(out[s][4],out[s][10])
axes[0][2].legend()

axes[0][0].set_xlabel('ell')
axes[0][0].set_ylabel('clkk')
axes[0][1].set_xlabel('ell')
axes[0][1].set_ylabel('clkk')
axes[0][2].set_xlabel('ell')
axes[0][2].set_ylabel('clkk')

axes[1][0].set_xlabel('kappa')
axes[1][1].set_xlabel('kappa')
axes[1][2].set_xlabel('kappa')
axes[1][0].set_ylabel('N_peaks')
axes[1][1].set_ylabel('N_minima')
axes[1][2].set_ylabel('PDF')

axes[2][0].set_xlabel('kappa')
axes[2][1].set_xlabel('kappa')
axes[2][2].set_xlabel('kappa')
axes[2][0].set_ylabel('V0')
axes[2][1].set_ylabel('V1')
axes[2][2].set_ylabel('V2')
plt.tight_layout()

In [ ]:
cosmo_tomo_cone_arr = [[cosmo, tomo, cone] 
                       for cosmo in cosmos 
                       for tomo in range(1,6)
                       for cone in range(1,6)]
print ( len(cosmo_tomo_cone_arr))

In [ ]:
run map_stats.py

In [ ]:
###### test if sigma_e -> sigma_kappa has a sqrt(2) factor
###### yes: sigma_kappa = sqrt(2) * sigma_e
### KS code from: https://lenstools.readthedocs.io/en/latest/_modules/lenstools/image/shear.html?highlight=kaiser%20squire#

# #Multipoles
lx = rfftfreq(smoothed_conv_maps[1][1].data.shape[0])[None]
ly = fftfreq(smoothed_conv_maps[1][1].data.shape[0])[:,None]
lsquared = lx**2 + ly**2
lsquared[0,0] = 1

#FFT forward, rotation, FFT backwards
conv_fft = rfft2(smoothed_conv_maps[1][1].data)
s1 = irfft2((lx**2-ly**2)*conv_fft/lsquared)
s2 = irfft2(2*lx*ly*conv_fft/lsquared)

In [ ]:
print ('kappa std', smoothed_conv_maps[1][1].std())
print ('shear std', s1.std(), s2.std())
print ('shear std * sqrt(2)',  sqrt(2)*s1.std(), sqrt(2)*s2.std())